# Class V - Fun visualizations in JuMP

This notebook contains a number of fun JuMP examples.

In [9]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


## Example 1: minimum bounding ellipse

Given a set of ellipses centered on the origin

$E(A) = \{ u\;|\;u^\top A^{-1} u <= 1 \}$

find a "minimal" ellipse that contains the provided ellipses

We can formulate this as an SDP:

$\begin{align}
     minimize \quad& trace(WX)\\
   subject to \quad& X \ge A_i, \quad i = 1,...,m \\
              &  X \succeq 0
\end{align}$

where $W$ is a positive-definite matrix of weights to choose between different solutions.

In [10]:
using JuMP, SCS, Plots, LinearAlgebra, Interact

function draw_ellipse(A::Matrix, args...; kwargs...)
    x_values = Float64[]
    y_values = Float64[]
    for angle in 0:0.001π:2π
        u = [cos(angle), sin(angle)]
        z = A * u
        push!(x_values, z[1])
        push!(y_values, z[2])
    end
    plot!(x_values, y_values, args...; kwargs...)
end

function solve_minimum_ellipse_problem(W, A_matrices)
    model = Model(solver = SCSSolver(eps = 1e-6, verbose = false))
    @variable(model, X[1:2, 1:2], SDP)
    @objective(model, Min, tr(W * X))
    for A in A_matrices
        @SDconstraint(model, X >= A)
    end
    status = solve(model)
    return status, JuMP.getvalue(X)
end

solve_minimum_ellipse_problem (generic function with 1 method)

Investigate the model. Here are some things to try:

- What happens if you comment out the first A matrix?

- What happens if you comment out the second A matrix?

- What happens if you comment out the third A matrix?

You can comment lines in Julia using the `#` symbol. As a shortcut, use `[CTRL] + [/]`.

In [11]:
@manipulate for weight in 1:20
    A_matrices = [
        [2.0 0.0; 0.0 1.0],
#         [1.0 0.0; 0.0 3.0],
        [2.3896 1.5433; 1.5433 1.35584]
    ]
    W = [1.0 0.0; 0.0 weight]
    status, X_value = solve_minimum_ellipse_problem(W, A_matrices)
    if status == :Optimal
        plot(legend = false)
        draw_ellipse.(A_matrices, color = "gray")
        draw_ellipse(X_value, color="purple", linewidth=2)
    else
        println("Could not solve. Status = $(status)")
    end
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-4ea4f0ce-b93c-40ba-947e-521767c4a336", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["weight"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row-left"), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>20,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-center"), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-right"), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget"), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Int64} with 2 listeners. Value:
10, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/carleton/.julia/packages/Knockout/JE2Yq/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/carleton/.julia/packages/Knockout/JE2Yq/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/carleton/.julia/packages/InteractBase/R8JgH/src/../assets/all.js"), Asset("css", nothing, "/Users/carleton/.julia/packages/InteractBase/R8JgH/src/../assets/style.css"), Asset("css", nothing, "/Users/carleton/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes"=>Any[JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"knockout-component-4ea4f0ce-b93c-40ba-947e-521767c4a336\",\"id\":\"ob_36\",\"type\":\"observable\"}),\"index\":WebIO.getval({\"name\":\"index\",\"scope\":\"knockout-component-4ea4f0ce-b93c-40ba-947e-521767c4a336\",\"id\":\"ob_35\",\"type\":\"observable\"})};\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n     

# Example 2: clustering points

See Section 2.3 of:
http://www.optimization-online.org/DB_FILE/2005/04/1114.pdf

Given 2-D data pairs $d_i$, $i=1,\ldots,N$, these points can be partitioned into $k$ clusters by solving the following SDP.

$\begin{align}
     minimize \quad& trace(W * (\mathbf{I} - X))\\
   subject\ to \quad& \Sigma_j X_{ij} = 1,\quad i=1, \ldots,N \\
              & trace(X) = k \\
              &  X \succeq 0
\end{align}$,

where $W_{ij} = e^{\frac{-||d_i - d_j||}{\sigma}}$.

In [12]:
""""
    calculate_weight_matrix(data::Matrix{Float64})

Calculates the distance between a list of 2-D data points given as
rows in `data` matrix.
"""
function calculate_weight_matrix(data::Matrix{Float64}, σ = 1.0)
    num_points = size(data, 1)
    W = zeros(num_points, num_points)
    for i in 1:num_points
        for j in (i+1):num_points
            dist = exp(-norm(data[i, :] - data[j, :]) / σ)
            W[i, j] = dist
            W[j, i] = dist
        end
    end
    return W
end

function eye(x, y)
    z = zeros(x, y)
    for i in 1:min(x, y)
        z[i, i] = 1.0
    end
    return z
end

function solve_cluster_problem(data::Matrix{Float64}, num_clusters::Int)
    W = calculate_weight_matrix(data)
    num_points = size(data, 1)
    
    model = Model(solver = SCSSolver(verbose = false))

    @variable(model, X[1:num_points, 1:num_points], SDP)
    @objective(model, Min, tr(W * (eye(num_points, num_points) - X)))
    @constraints(model, begin
        X .>= 0
        [i in 1:num_points], sum(X[i, :]) == 1
        tr(X) == num_clusters
    end)

    status = solve(model)

    X_value = getvalue(X)

    cluster = zeros(Int, num_points)
    cluster_counter = 0
    for i in 1:num_points
        if cluster[i] == 0
            cluster_counter += 1
            cluster[i] = cluster_counter
            for j in (i+1):num_points
                if norm(X_value[i, j] - X_value[i, i]) <= 1e-6
                    cluster[j] = cluster[i]
                end
            end
        end
    end
    return cluster
end

solve_cluster_problem (generic function with 1 method)

Investigate the model. What goes wrong when?

In [13]:
data = vcat(
    rand(Float64, (10, 2)) .+ [2 3],
    rand(Float64, (10, 2)) .+ [4 6],
    rand(Float64, (10, 2)) .+ [3.5 3]
)

@manipulate for num_clusters = 1:4
    which_clusters = solve_cluster_problem(data, num_clusters)
    plot(
        xlabel = "x", ylabel = "y", 
        xlims=(0, 8), ylims = (0, 8),
        legend = :bottomright
    )
    for k in 1:maximum(which_clusters)
        points = which_clusters .== k
        scatter!(data[points, 1], data[points, 2], 
            label = "Cluster $(k)", markersize=10)
    end
    plot!()
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-83624529-fe85-49fe-bd52-a9094c418d11", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["num_clusters"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row-left"), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>4,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-center"), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-right"), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget"), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Int64} with 2 listeners. Value:
2, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/carleton/.julia/packages/Knockout/JE2Yq/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/carleton/.julia/packages/Knockout/JE2Yq/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/carleton/.julia/packages/InteractBase/R8JgH/src/../assets/all.js"), Asset("css", nothing, "/Users/carleton/.julia/packages/InteractBase/R8JgH/src/../assets/style.css"), Asset("css", nothing, "/Users/carleton/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes"=>Any[JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"knockout-component-83624529-fe85-49fe-bd52-a9094c418d11\",\"id\":\"ob_44\",\"type\":\"observable\"}),\"index\":WebIO.getval({\"name\":\"index\",\"scope\":\"knockout-component-83624529-fe85-49fe-bd52-a9094c418d11\",\"id\":\"ob_43\",\"type\":\"observable\"})};\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : k

## Example 3 - Ridge regression

In this example, we consider fitting a quadratic function to a noisy signal using [ridge regression](https://en.wikipedia.org/wiki/Tikhonov_regularization).

$\begin{align}
      minimize \quad& ||z - y||_2 + \lambda||\theta||_2\\
   subject\ to \quad& z_i = \theta_1 x_i^2 + \theta_2 x_i + \theta_3
\end{align}$,

In [14]:
using Ipopt

function solve_ridge_regression(x, y, λ)            
    N = length(x)
    model = Model(solver = IpoptSolver(print_level = 0))
    @variable(model, θ[i = 1:4], start = 1.0)
    @variable(model, z[i = 1:N], start = x[i]^2)
    @constraint(model, [i = 1:N], z[i] == θ[1] + θ[2] * x[i] + θ[3] * x[i]^2+ θ[4] * x[i]^3)
    @NLobjective(model, Min, 
        sum((z[i] - y[i])^2 for i in 1:N) + λ * sum(θ[i]^2 for i in 1:4)
    )
    solve(model)
    return JuMP.getvalue(z)
end

solve_ridge_regression (generic function with 1 method)

Play round with the weight in front of the regularizer term. What changes? What happens if you increase the number of data points?

In [15]:
N = 10
x = rand(N)
sort!(x)
y = 5 * x.^2 .+ x
yϵ = y .+ (1.5 * rand(N) .- 0.5)

@manipulate for λ in 0:0.01:0.25
    z = solve_ridge_regression(x, yϵ, λ)
    plot(x, y, linestyle=:dash, color="purple")
    scatter!(x, yϵ, 
        legend=false, color="gray", markerstrokewidth=0
    )
    plot!(x, z, linewidth=2, color="purple")
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-25549ea5-7988-4d5a-a4a9-c5a2a5fbcbca", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["λ"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row-left"), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>26,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-center"), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-right"), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget"), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Any} with 2 listeners. Value:
13, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/carleton/.julia/packages/Knockout/JE2Yq/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/carleton/.julia/packages/Knockout/JE2Yq/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/carleton/.julia/packages/InteractBase/R8JgH/src/../assets/all.js"), Asset("css", nothing, "/Users/carleton/.julia/packages/InteractBase/R8JgH/src/../assets/style.css"), Asset("css", nothing, "/Users/carleton/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes"=>Any[JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"0.01\",\"0.02\",\"0.03\",\"0.04\",\"0.05\",\"0.06\",\"0.07\",\"0.08\",\"0.09\",\"0.1\",\"0.11\",\"0.12\",\"0.13\",\"0.14\",\"0.15\",\"0.16\",\"0.17\",\"0.18\",\"0.19\",\"0.2\",\"0.21\",\"0.22\",\"0.23\",\"0.24\",\"0.25\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"knockout-component-25549ea5-7988-4d5a-a4a9-c5a2a5fbcbca\",\"id\":\"ob_53\",\"type\":\"observable\"}),\"index\":WebIO.getval({\"name\":\"index\",\"scope\":\"knockout-component-25549ea5-7988-4d5a-a4a9-c5a2a5fbcbca\",\"id\":\"ob_52\",\"type\":\"observable\"})};\n   

# Example 4 - feasible regions

This example explores a problem with a nonlinear feasible region.

In [16]:
@manipulate for λ in 0.05:0.05:0.95
    model = Model(solver = IpoptSolver(print_level = 0))
    @variable(model, 0 <= x <= 20)
    @variable(model, 0 <= y <= 20)
    @NLconstraint(model, y >= 0.05x^2)
    @NLconstraint(model, y >= exp(-0.25 * (x - 10)))
    @objective(model, Min, λ * x + (1 - λ) * y)
    solve(model)

    b = 1
    a = b * (λ - 1) / λ
    ab = a^2 + b^2
    b /= ab
    a /= ab

    x_value = JuMP.getvalue(x)
    y_value = JuMP.getvalue(y)

    z = -5:0.1:25.0
    plot(xlims=(-2.5, 10), ylims=(0, 15), legend=false)
    vline!([0.0], color="gray", linewidth=2)
    plot!(z, 0.05z.^2, color="gray", linewidth=2)
    plot!(z, exp.(-0.25(z.-10)), color="gray", linewidth=2)
    scatter!([JuMP.getvalue(x)], [JuMP.getvalue(y)], color="purple", markersize=10)
    plot!(z, y_value .+ (b / a) * (z .- x_value), linestyle=:dash, color="purple")
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-765d1878-9307-4756-bb12-617dd45fdae4", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["λ"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row-left"), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>19,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-center"), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-right"), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget"), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Any} with 2 listeners. Value:
10, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/carleton/.julia/packages/Knockout/JE2Yq/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/carleton/.julia/packages/Knockout/JE2Yq/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/carleton/.julia/packages/InteractBase/R8JgH/src/../assets/all.js"), Asset("css", nothing, "/Users/carleton/.julia/packages/InteractBase/R8JgH/src/../assets/style.css"), Asset("css", nothing, "/Users/carleton/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes"=>Any[JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.05\",\"0.1\",\"0.15\",\"0.2\",\"0.25\",\"0.3\",\"0.35\",\"0.4\",\"0.45\",\"0.5\",\"0.55\",\"0.6\",\"0.65\",\"0.7\",\"0.75\",\"0.8\",\"0.85\",\"0.9\",\"0.95\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"knockout-component-765d1878-9307-4756-bb12-617dd45fdae4\",\"id\":\"ob_62\",\"type\":\"observable\"}),\"index\":WebIO.getval({\"name\":\"index\",\"scope\":\"knockout-component-765d1878-9307-4756-bb12-617dd45fdae4\",\"id\":\"ob_61\",\"type\":\"observable\"})};\n    var self = this;\n    function AppViewModel() {\n        for (var ke